In [33]:
import pandas as pd
import itertools

In [34]:
dataset = pd.read_excel("CoffeeShopTransactions.xlsx")

In [35]:
sup = int(input("Enter Minimum support (integer): "))
input_conf = float(input("Enter Minimum confidence (double): "))

Enter Minimum support (integer): 2
Enter Minimum confidence: 0.5


In [36]:
dataset.head()

,Date,Time,Transaction Number,Item 1,Item 2,Item 3
0,2016-10-30,10:07:57,1,Hot chocolate,CaramelBites,Mineral water
1,2016-10-30,10:07:57,2,Cookies,CaramelBites,CaramelBites
2,2016-10-30,10:13:03,3,Coffee,Chocolates,Brownie
3,2016-10-30,10:19:12,4,Coffee,Brownie,Tea
4,2016-10-30,10:19:12,5,Tea,Cookies,Cake


In [37]:
items = dataset[['Item 1','Item 2','Item 3']].copy()
items.head()

,Item 1,Item 2,Item 3
0,Hot chocolate,CaramelBites,Mineral water
1,Cookies,CaramelBites,CaramelBites
2,Coffee,Chocolates,Brownie
3,Coffee,Brownie,Tea
4,Tea,Cookies,Cake


In [38]:
vertical_data={}

In [40]:
for i in range(items.shape[0]):
    for j in range(3):
        if(vertical_data.get(items.iloc[i][j])):
            vertical_data[items.iloc[i][j]].append(str(i))#make it string
        else:
            vertical_data[items.iloc[i][j]] = [str(i)]

In [41]:
sup_count = {}
for item in vertical_data:
    sup_count[item] = len(vertical_data[item])

In [43]:
def intersect_list(l1,l2):
    return list(set(l1)&set(l2))

In [44]:
def list_def(a,b):
    return list(set(a)-set(b))

In [45]:
def list_to_str(l):
    l_list = list(l)
    l_list.sort()
    return ','.join(l_list)

In [46]:
def str_to_list(s):
    return s.split(',')

In [47]:
new_vertical_data = {}
all_key_list = []
frequent_item_set = {}
key_set = set()
iteration = 0
while 1:
    iteration+=1
    vertical_data_copy = vertical_data.copy()
    for item in vertical_data_copy:
        if(len(vertical_data_copy[item])<sup):
            del vertical_data[item]
    all_key_list = list(vertical_data.keys())
    for i in range(len(all_key_list)):
        for j in range(len(all_key_list)):
            if(j>i):
                key_set.update(str_to_list(all_key_list[i]))
                key_set.update(str_to_list(all_key_list[j]))
                if(len(key_set)==iteration+1):
                    key_set_str = list_to_str(key_set)
                    new_vertical_data[key_set_str] = intersect_list(vertical_data[all_key_list[i]] , vertical_data[all_key_list[j]])
                    sup_count[key_set_str] =  len(new_vertical_data[key_set_str])
                key_set = set()

    con = False
    for item in new_vertical_data:
        if(len(new_vertical_data[item])>=sup):
            vertical_data = new_vertical_data
            new_vertical_data = {}
            con = True
            break
    if(not con):
        frequent_item_set = vertical_data
        break
    

In [48]:
frequent_item_set_keys = list(frequent_item_set.keys())
strong_association_rules = []

In [77]:
print("+"*57)
print("+"*10 + " "*10 +  "Frequent item sets" + " "*10 + "+"*9)
print("+"*57)
for frequent_item in frequent_item_set_keys:
    print("+ " + frequent_item + " "*(54-len(frequent_item)) + "+")
print("+"*57)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++          Frequent item sets          +++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+ CaramelBites,Hot chocolate,Mineral water              +
+ CaramelBites,Cookies,Hot chocolate                    +
+ CaramelBites,Coffee,Hot chocolate                     +
+ CaramelBites,Chocolates,Hot chocolate                 +
+ Brownie,CaramelBites,Hot chocolate                    +
+ CaramelBites,Hot chocolate,Tea                        +
+ Cake,CaramelBites,Hot chocolate                       +
+ CaramelBites,Hot chocolate,Juice                      +
+ Cookies,Hot chocolate,Mineral water                   +
+ Coffee,Hot chocolate,Mineral water                    +
+ Chocolates,Hot chocolate,Mineral water                +
+ Brownie,Hot chocolate,Mineral water                   +
+ Hot chocolate,Mineral water,Tea                       +
+ Cake,Hot chocolate,Mineral water                      +
+ Hot chocolat

In [50]:
for frequent_item in frequent_item_set_keys:# 'c,a' in ['c,a', 'd,a', 'c,b', 'd,c', 'x,d']
    for i in range(len(str_to_list(frequent_item))): #[a,b,c,d] i.e.: 0,1,2,3 we need 1,2 
        if(i==0):
            continue
        for item_combination in itertools.combinations(str_to_list(frequent_item), i):# [a,b,c] -> ('a', 'b') , ('a', 'c') , ...
            str_item_combination_A = list_to_str(list(item_combination))# ('a', 'b') -> 'a,b'
            str_item_combination_B = list_to_str(list_def(str_to_list(frequent_item),list(item_combination)))
            conf = sup_count[frequent_item] / sup_count[str_item_combination_A]
            if(conf >= input_conf):
                strong_association_rules.append(str_item_combination_A + " ==> " + str_item_combination_B)
            

In [51]:
strong_association_rules

['Caramel bites ==> Hot chocolate,Tea',
 'Caramel bites,Hot chocolate ==> Tea',
 'Caramel bites,Tea ==> Hot chocolate',
 'CaramelBites,Chocolates ==> Coffee']

In [79]:
print("*"*57)
print("*"*10 + " "*10 +  "Association rules" + " "*10 + "*"*10)
print("*"*57)
for association_rule in strong_association_rules:
    print("* " + association_rule + " "*(54-len(association_rule)) + "*")
print("*"*57)

*********************************************************
**********          Association rules          **********
*********************************************************
* Caramel bites ==> Hot chocolate,Tea                   *
* Caramel bites,Hot chocolate ==> Tea                   *
* Caramel bites,Tea ==> Hot chocolate                   *
* CaramelBites,Chocolates ==> Coffee                    *
*********************************************************
